# Strands Agentsセッション間でのメモリの永続化

この例では、Strands Agentsのさまざまなセッション間でメモリを永続化する方法を学びます。

duckduckgo検索APIを使用してWeb検索を行うエージェントのユースケースを使用します。

このノートブックでは、次のことを行います：

- メモリを搭載したStrands agentの機能を探索します
- メモリの保存、取得、リスト方法を学びます
- エージェントを介してWeb検索を実行する方法を理解します
- インタラクティブループでエージェントとやり取りします


### 使用例

メモリの保存：

Remember that I prefer tea over coffee

メモリの取得：

What do I prefer to drink?

すべてのメモリのリスト：

Show me everything you remember about me

### メモリ使用のヒント

- エージェントに情報を記憶させるときは明示的に指示してください
- 関連するメモリを取得するには具体的なクエリを使用してください
- メモリの永続化により、より自然で文脈に沿った会話が可能になります

## セットアップと前提条件

### 前提条件

- Python 3.10以上
- AWSアカウントと環境で設定されたAWS認証情報
- Amazon BedrockでAnthropic Claude 3.7が有効化されていること
- Amazon Bedrock Knowledge Base、Amazon S3バケット、Amazon DynamoDBを作成する権限を持つIAMロール

それでは、Strands Agentに必要なパッケージをインストールしましょう

In [ ]:
# Install the required packages
!pip install -r requirements.txt

In [ ]:
# Import Required Libraries
import os
from strands import Agent, tool
from strands.models import bedrock
from strands_tools import mem0_memory
import time

from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException

os.environ["AWS_REGION"] = os.environ.get("AWS_REGION", "us-east-1")
bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## Mem0設定

### メモリバックエンドオプション

Mem0 Memory Toolは3つの異なるバックエンド設定をサポートしています：

1. **[OpenSearch](https://aws.amazon.com/opensearch-service/features/serverless/)** (AWS環境で推奨)：
   - AWS認証情報とOpenSearch設定が必要です
   - OPENSEARCH_HOSTを設定し、オプションでAWS_REGION（デフォルトはus-west-2）を設定します

2. **[FAISS](https://faiss.ai/index.html)** (ローカル開発のデフォルト)：
   - FAISSをローカルベクトルストアバックエンドとして使用します
   - ローカルベクトルストレージにはfaiss-cpuパッケージが必要です
   - 追加の設定は不要です

3. **Mem0 Platform**：
   - [Mem0 Platform API](https://docs.mem0.ai/platform/quickstart)を使用してメモリ管理を行います
   - 環境変数にMEM0_API_KEYが必要です


### 環境設定

| 環境変数 | 説明 | デフォルト | 必須対象 |
|---------------------|-------------|----------|--------------|
| OPENSEARCH_HOST | OpenSearch ServerlessホストURL | なし | OpenSearch |
| AWS_REGION | OpenSearchのAWSリージョン | us-west-2 | OpenSearch |
| MEM0_API_KEY | Mem0 Platform APIキー | なし | Mem0 Platform |
| DEV | 開発モードを有効にする | false | すべてのモード |


このラボの範囲では、メモリ管理のバックエンドとして2つのオプションを使用できます：

### オプション1. [Opensearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/)

これがAOSSのセットアップアーキテクチャです：

<div style="text-align:left">
    <img src="images/arch_AOSS.png" width="65%" />
</div>

In [ ]:
# You can manually define your Opensearch Host 
# os.environ["OPENSEARCH_HOST"] = "<your-opensearch-host>.<region>.aoss.amazonaws.com"

In [ ]:
# OR - Run the script to Create Opensearch Serverless resource in your AWS Account
!bash prereqs/deploy_OSS.sh

In [ ]:
wait_time = 120
print(f"Waiting {wait_time} seconds for OpenSearch Serverless collection to be fully operational...")
time.sleep(wait_time)

In [ ]:
# Option 1: Opensearch Serverless
from dotenv import load_dotenv
load_dotenv()


### オプション2 [Mem0 Platform](https://docs.mem0.ai/platform):

#### [注意]: すでにOpensearch Serverlessオプションをデプロイしている場合、これは不要です。

代わりに、[こちら](https://docs.mem0.ai/platform/quickstart#2-api-key-setup)の手順に従ってMem0 APIキーを作成し、環境変数**MEM0_API_KEY**として追加できます。

これがMem0 Platformのセットアップアーキテクチャです：

<div style="text-align:left">
    <img src="images/arch_mem0.png" width="65%" />
</div>


エージェントの機能を有効にするには、AWS認証情報とOpenSearch/Mem0 Platformの環境変数を設定する必要があります。これらの変数はメモリの保存と取得に使用されます。

In [ ]:
# Option 2: Mem0 API key
# os.environ["MEM0_API_KEY"] = "<your-mem0-api-key>"

## システムプロンプトの定義

SYSTEM_PROMPT変数は、メモリエージェントの動作と機能を定義します。このプロンプトは、保存されたメモリに基づいてパーソナライズされた応答を提供し、必要に応じてWeb検索を実行するようにエージェントをガイドします。

In [ ]:
# Define a focused system prompt for memory operations
SYSTEM_PROMPT = """You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
"""

## Web検索ツールの定義

[Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search)を使用したwebsearchツール関数により、エージェントがWeb検索を実行できます。この関数は例外を処理し、検索結果または適切なエラーメッセージを返します。

In [ ]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## メモリエージェントの作成

定義したツールとシステムプロンプトを使用して、メモリに焦点を当てたエージェントを初期化します。Strands agentは次のことができます：

1. コンテキストに基づいてメモリを保存および取得します。メモリを使用して、よりパーソナライズされた文脈に沿ったAIインタラクションを作成します。
2. DuckDuckGoを使用してWeb検索を実行し、最新情報を提供します。

In [ ]:
# Create an agent with memory, websearch tool
USER_ID = "new_user"

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[mem0_memory, websearch],
)

## メモリ操作のデモンストレーション

以下の例は、メモリエージェントを使用してメモリを保存、取得、リストする方法を示しています。

- **store**: 後で取得するために重要な情報を保存
  - ユーザー設定を保存
  - 重要な事実を記憶
  - 会話のコンテキストを維持

- **retrieve**: クエリに基づいて関連するメモリにアクセス
  - 以前に保存された情報を検索
  - ユーザー履歴に基づいてパーソナライズされた応答を提供

- **list**: 保存されたすべてのメモリを表示
  - どのような情報が保持されているかを確認
  - 保存されたメモリを監査

In [ ]:
# Store initial memories to demonstrate retrieval
memory_agent.tool.mem0_memory(
    action="store", content=f"The user's name is {USER_ID}.", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="I like to drink tea more than coffee.", 
    user_id=USER_ID
)

In [ ]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="What is the user's name?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

In [ ]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="What are the my drink preferences?",
    user_id=USER_ID
)

In [ ]:
# Ask the agent a question
response = memory_agent("What are the events happening in the New York today?")
print(response)

In [ ]:
# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

## インタラクティブなエージェントの使用

最後に、ユーザーがメモリエージェントとやり取りするためのインタラクティブループを提供します。ユーザーは新しいメモリを保存したり、既存のメモリを取得したり、保存されたすべてのメモリをリストしたりできます。

インタラクティブな使用をテストするには：

1. 要件をインストールします：`pip install -r requirements.txt`
2. pythonファイル`personal_agent_with_memory.py`を実行します。

## 結論

このノートブックでは、Strandsフレームワークを使用してメモリ機能を持つパーソナルエージェントを作成する方法を示しました。エージェントは次のことができます：

1. ユーザーに関する情報を保存
2. コンテキストに基づいて関連するメモリを取得
3. 追加情報のためにWebを検索
4. パーソナライズされた応答を提供

これらの機能を組み合わせることで、エージェントは会話全体でコンテキストを維持し、時間の経過とともによりパーソナライズされた支援を提供できます。

### クリーンアップ

このBashスクリプトを実行して、Opensearch Serverlessリソースをクリーンアップします。MEM0_PLATFORM_APIを使用した場合は実行不要です。

In [ ]:
!sh prereqs/cleanup_OSS.sh